<a href="https://colab.research.google.com/github/Jenniferduong20012004/JennyLearningAI/blob/main/LLMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#LLMS
#computer vision: dữ liệu đầu vào là ảnh và video
#natural language processing: văn bản text
#time series: dự báo thời tiết, chứng khoáng

In [ ]:
#build vietnamese chatbox can handle daily conservation and instruction using llms

In [3]:
#install library
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U googletrans==3.1.0a0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import(
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
os.environ ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
#load pretrain model

MODEL_NAME = "vilm/vinallama-7b-chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config #allow to work on gg collab
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#configurate llms
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training (model)
config = LoraConfig(
    r = 16,
    lora_alpha =32,
    target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CASUAL_LM"
)
model = get_peft_model (model, config)

In [7]:
generation_config = model.generation_config
generation_config.amx_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [8]:
#prompt
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Viết một hàm Python tính tổng 3 số.
<|im_end|>
<|im_start|>assistant
""".strip()

In [9]:
#run pretrained model
%%time
from IPython.display import Javascript
display (Javascript ('''google.colab.output.setIframeHeight(0,true, {maxHeight:500})'''))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoding = tokenizer (prompt, return_tensors = 'pt').to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids= encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )
print (tokenizer.decode(outputs[0], skip_special_tokens = True))

<IPython.core.display.Javascript object>

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Viết một hàm Python tính tổng 3 số.
 
<|im_start|> assistant
Dưới đây là một hàm Python nhận ba số làm đầu vào và trả về tổng của chúng:

```python
def sum_three_numbers(a, b, c):
    return a + b + c

result = sum_three_numbers(5, 7, 8)
print(result)  # Kết quả: 21
```

Hàm `sum_three_numbers` nhận ba tham số: `a`, `b`, và `c`. Nó sử dụng toán tử `+` để cộng ba số này lại và trả về tổng. Trong ví dụ này, hàm được gọi với các số 5, 7, và 8 làm đối số, và kết quả được lưu trong biến `result`. Cuối cùng, kết quả được in ra màn hình console.
CPU times: user 24.2 s, sys: 744 ms, total: 24.9 s
Wall time: 27.2 s


In [10]:
#download dataset
data_alespalla = load_dataset ('alespalla/chatbot_instruction_prompts')

Generating train split:   0%|          | 0/258042 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/64511 [00:00<?, ? examples/s]

In [13]:
data_alespalla

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})

In [12]:
data_alespalla['train'][70]

{'response': "You don't enjoy talking to him and that's fine but don't you think that's a bit extreme? You could just tell him to leave you alone or something instead. Doing something extreme to sold such a small problem isn't the right play.",
 'prompt': 'He was talking to me about boring things and I wanted to get rid of him.'}

In [14]:
#prepare training dataset need output answer
#change from eng to vietnamese --> use gg translate
def generate_prompt (data_point):
  translator = Translator()
  vn_prompt = translator.translate (data_point['prompt'], src = 'en', dest = 'vi').text
  vn_response = translator.translate (data_point['response'], src = 'en', dest = 'vi').text
  return f"""
  <|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
{vn_prompt}
<|im_end|>
<|im_start|>assistant
{vn_response}
""".strip()

In [15]:
def generate_and_tokenize_prompt (data_point):
  full_prompt = generate_prompt (data_point)
  tokenized_full_prompt = tokenizer (
      full_prompt,
      padding = True,
      truncation = True
  )
  return tokenized_full_prompt

In [16]:
data_alespalla = data_alespalla['train'].shard (num_shards = 50, index =0).filter(
    lambda sample: sample ['response']!= '' and sample ['prompt'] != ''
).shuffle().map(generate_and_tokenize_prompt)

Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
#training
training_args = transformers.TrainingArguments (
    per_device_train_batch_size =1,
    gradient_accumulation_steps = 4,
    num_train_epochs =1,
    learning_rate = 2e-4,
    fp16 = True,
    save_total_limit = 3,
    logging_steps =1,
    output_dir = "experiments",
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.05
)
trainer = transformers.Trainer(
    model = model,
    train_dataset = data_alespalla,
    args = training_args,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.814500
2,3.985100
3,4.533400
4,4.660800
5,3.639600
6,3.193000
7,3.628700
8,3.281500
9,3.737100
10,3.752000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
